># 1. 데이터 전처리

- feature engineering 편의성을 위해 test셋을 train 폴더에 복사해서 전처리를 한번에 하였음

In [1]:
import pandas as pd
import numpy as np
import glob
import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
tmp_data = pd.read_csv("./train/ace_2000.csv")
tmp_data.head(20)
del tmp_data
## 12번 row까지는 생략이 필요함

In [3]:
##데이터 길이 확인
file_name = glob.glob("./train/ace_"+"*.csv")
for file in file_name:
    print(file+' 의 데이터 길이 : '+str(len(pd.read_csv(file))))
    
del file_name
## 결과로 결측값이 존재하는 것을 확인함

./train\ace_1999.csv 의 데이터 길이 : 492764
./train\ace_2000.csv 의 데이터 길이 : 494114
./train\ace_2001.csv 의 데이터 길이 : 492766
./train\ace_2002.csv 의 데이터 길이 : 492764
./train\ace_2003.csv 의 데이터 길이 : 492763
./train\ace_2004.csv 의 데이터 길이 : 494113
./train\ace_2005.csv 의 데이터 길이 : 492764
./train\ace_2006.csv 의 데이터 길이 : 492764
./train\ace_2007.csv 의 데이터 길이 : 492764
./train\ace_2008.csv 의 데이터 길이 : 494112
./train\ace_2009.csv 의 데이터 길이 : 492759
./train\ace_2010.csv 의 데이터 길이 : 525613
./train\ace_2011.csv 의 데이터 길이 : 492762
./train\ace_2012.csv 의 데이터 길이 : 494113
./train\ace_2013.csv 의 데이터 길이 : 492764


In [4]:
all_data = list()
for year in range(1999,2014) :
    print(year, '데이터 csv 형식으로 변경 완료')
    tmp_df = pd.read_csv("./train/ace_"+str(year)+".csv")
    tmp_df = tmp_df[11:]
    col_names = list(map(lambda x: x.split(), list(tmp_df[:1].values[0])))[0]
    tmp_df = tmp_df[2:]
    splited_rows = list(map(lambda x: x.split(),tmp_df[tmp_df.columns[0]][2:]))
    new_df = pd.DataFrame(splited_rows,columns=col_names)
    new_df['date'] = new_df['doy'].apply(lambda x : datetime.date(int(new_df.year[0])-1,12,31) + datetime.timedelta(days = int(x)))
    new_df['hr'] = new_df['hr'].apply( lambda x : str(x).zfill(2) )
    new_df['min'] = new_df['min'].apply( lambda x : str(x).zfill(2) )
    new_df['date'] = new_df['date'].apply(lambda x : str(x))
    new_df['date'] += ' ' + new_df['hr']
    new_df['date'] += ':' + new_df['min']
    all_data.append(new_df)

all_df = pd.concat(all_data)
# del all_data, new_df, splited_rows, tmp_df, col_names

# all_df.to_csv("./train/train_new.csv",  index=False)
# train_data = pd.read_csv("./train/train_x.csv")
## 저장하고 다시 불러오면 object가 float로 변하는 등 원래 데이터 타입을 찾아감

1999 데이터 csv 형식으로 변경 완료
2000 데이터 csv 형식으로 변경 완료
2001 데이터 csv 형식으로 변경 완료
2002 데이터 csv 형식으로 변경 완료
2003 데이터 csv 형식으로 변경 완료
2004 데이터 csv 형식으로 변경 완료
2005 데이터 csv 형식으로 변경 완료
2006 데이터 csv 형식으로 변경 완료
2007 데이터 csv 형식으로 변경 완료
2008 데이터 csv 형식으로 변경 완료
2009 데이터 csv 형식으로 변경 완료
2010 데이터 csv 형식으로 변경 완료
2011 데이터 csv 형식으로 변경 완료
2012 데이터 csv 형식으로 변경 완료
2013 데이터 csv 형식으로 변경 완료


In [5]:
del all_data, new_df, splited_rows, tmp_df, col_names

all_df.to_csv("./train/train_new.csv",  index=False)
train_data = pd.read_csv("./train/train_new.csv")
# 저장하고 다시 불러오면 object가 float로 변하는 등 원래 데이터 타입을 찾아감

In [6]:
for col in list(train_data.columns)[4:-1]:
    print(train_data[col][train_data[col] <= -9999].unique())
### 이를 통해 NaN 값은 -9999.9와 -9999.9004가 존재하는 것을 확인

[-9999.9    -9999.9004]
[-9999.9    -9999.9004]
[-9999.9    -9999.9004]
[-9999.9    -9999.9004]
[-9999.9    -9999.9004]
[-9999.9    -9999.9004]
[-9999.9    -9999.9004]


In [7]:
train_data = train_data.replace(-9999.9 , np.nan)
train_data = train_data.replace(-9999.9004 , np.nan)
## np.nan으로 변경

train_data['date'] = pd.to_datetime(train_data['date'])
## data market과 join하기 위해 datetime 형식으로 변경

In [8]:
## data market을 생성하고 기존 train data를 merge
## train data 확인 결과 1999~2012년 까지 데이터가 존재하는 것으로 확인됨

train_market = list()
for year in range(1999, 2014):
    tmp_date = pd.date_range(start='01/01/'+str(year), end='12/31/'+str(year))
    tmp_df_date = pd.DataFrame(tmp_date, columns=['date'])
    tmp_df_date['date'] = tmp_df_date['date'].astype(str)
    tmp_df_date['date'] = tmp_df_date['date'].str[0:10]
    
    tmp_time = pd.date_range(start='00:00:00', end='23:59:00', freq='1min')
    tmp_df_time = pd.DataFrame(tmp_time, columns=['time'])
    tmp_df_time['time'] = tmp_df_time['time'].astype(str)
    tmp_df_time['time'] = tmp_df_time['time'].str[10:19]
    
    tmp_init_DT = pd.DataFrame(columns=['date', 'time'])
    tmp_init_DT2 = pd.DataFrame(columns=['date', 'time'])
    
    for i in range(len(tmp_date)):
        if i == 0:
            tmp_init_DT['time'] = tmp_df_time['time']
            tmp_init_DT['date'] = tmp_df_date['date'][0]
        else:
            tmp_init_DT2['time'] = tmp_df_time['time']
            tmp_init_DT2['date'] = tmp_df_date['date'][i]
            tmp_init_DT = tmp_init_DT.append(tmp_init_DT2)
    
    train_market.append(tmp_init_DT)
    print('%d data market 생성완료' %year)

del tmp_df_time, tmp_df_date, tmp_init_DT2, tmp_init_DT

train_market = pd.concat(train_market)

1999 data market 생성완료
2000 data market 생성완료
2001 data market 생성완료
2002 data market 생성완료
2003 data market 생성완료
2004 data market 생성완료
2005 data market 생성완료
2006 data market 생성완료
2007 data market 생성완료
2008 data market 생성완료
2009 data market 생성완료
2010 data market 생성완료
2011 data market 생성완료
2012 data market 생성완료
2013 data market 생성완료


In [9]:
train_market['time_conv'] = train_market['date'] + train_market['time']
train_market = train_market.drop(columns=['date', 'time'])

train_data = train_data.rename(columns={'date':'time_conv'})
## join을 위해 변경

train_market['time_conv'] = pd.to_datetime(train_market['time_conv'])
## join을 위해 변경

join_result = pd.merge(train_market, train_data, how = 'left', on = 'time_conv')
## 1분 단위로 모든 시간이 있는 data market에 join하여 nan값 전처리 준비

join_result.to_csv("./train/train_1min.csv",  index=False)
join_result = pd.read_csv("./train/train_1min.csv")
## 저장 후 불러와서 자료형 일치 시킴

join_result = join_result.drop(columns=['year', 'doy', 'hr', 'min'])
join_result['time_conv'] = pd.to_datetime(join_result['time_conv'])
tmp_data_10min = join_result.set_index('time_conv')
tmp_data_10min = tmp_data_10min.resample('10Min').mean()

del join_result

## 10분 단위 평균으로 데이터셋 변경

tmp_data_10min = tmp_data_10min.interpolate(method='linear')
tmp_data_10min.isnull().sum()
## 선형 interpolation으로 nan 제거

Np         0
Tp         0
Vp         0
B_gsm_x    0
B_gsm_y    0
B_gsm_z    0
Bt         0
dtype: int64

In [10]:
## 3시간 단위로 resampling

tmp_data_3H_mean = tmp_data_10min.resample('3H').mean()
tmp_data_3H_max = tmp_data_10min.resample('3H').max()
tmp_data_3H_min = tmp_data_10min.resample('3H').min()
tmp_data_3H_std = tmp_data_10min.resample('3H').std()
tmp_data_3H_var = tmp_data_10min.resample('3H').var()

train_data_3H_col = list()
for str in ['mean_', 'max_', 'min_', 'std_', 'var_']:
    train_data_3H_col.append(list(map(lambda x: str+x, list(tmp_data_10min.columns))))

tmp_data_3H_mean.columns = train_data_3H_col[0]
tmp_data_3H_max.columns = train_data_3H_col[1]
tmp_data_3H_min.columns = train_data_3H_col[2]
tmp_data_3H_std.columns = train_data_3H_col[3]
tmp_data_3H_var.columns = train_data_3H_col[4]

tmp_data_3H_mean.to_csv('tmp_data_3H_mean.csv',index=False)
tmp_data_3H_max.to_csv('tmp_data_3H_max.csv',index=False)
tmp_data_3H_min.to_csv('tmp_data_3H_min.csv',index=False)
tmp_data_3H_std.to_csv('tmp_data_3H_std.csv',index=False)
tmp_data_3H_var.to_csv('tmp_data_3H_var.csv',index=False)

tmp_train0 = pd.DataFrame(tmp_data_3H_mean.index, columns=['time_conv'])
tmp_train1 = pd.read_csv('tmp_data_3H_mean.csv')
tmp_train2 = pd.read_csv('tmp_data_3H_max.csv')
tmp_train3 = pd.read_csv('tmp_data_3H_min.csv')
tmp_train4 = pd.read_csv('tmp_data_3H_std.csv')
tmp_train5 = pd.read_csv('tmp_data_3H_var.csv')

train_fin = pd.concat([tmp_train0, tmp_train1, tmp_train2, tmp_train3, tmp_train4, tmp_train5], axis=1)
del tmp_train0, tmp_train1, tmp_train2, tmp_train3, tmp_train4, tmp_train5

tmp_test = pd.read_csv("Kp_index.csv")
kp_list = list()
for i in range(len(tmp_test.values)):
    kp_list += list(tmp_test.values[i][1:])

In [11]:
train_fin.to_csv('train_fin.csv',index=False)
train_fin = pd.read_csv('train_fin.csv')

tmp_df = pd.DataFrame(train_fin['time_conv'].str.split(' ').tolist(),columns = ['date','time'])
tmp_df2 = pd.DataFrame(tmp_df['date'].str.split('-').tolist(),columns = ['year','month', 'day'])
tmp_df3 = pd.DataFrame(tmp_df['time'].str.split(':').tolist(),columns = ['hour','min', 'sec'])
tmp_df3 = tmp_df3.drop(columns=['sec','min'])
train_fin = train_fin.drop(columns=['time_conv'])
tmp_df4= pd.concat([tmp_df2, tmp_df3], axis=1)
train_fin = pd.concat([tmp_df4, train_fin], axis=1)
del tmp_df, tmp_df2, tmp_df3, tmp_df4

In [12]:
test_fin = train_fin[train_fin['year'] == '2013']
train_fin = train_fin[train_fin['year'] != '2013']

In [13]:
train_fin['target'] = kp_list
del kp_list, tmp_test

## 예측해야하는 목표값인 kp index 파일을 3시간 단위로 만든 train데이터에 붙이는 작업 실시
## train data에 kp를 붙여서 예측할 target 까지 포함한 데이터셋으로 제작

train_fin.to_csv('train_fin.csv',index=False)
test_fin.to_csv('test_fin.csv',index=False)
del train_fin, test_fin
###완성파일 보존

> # 2. 모델생성

In [14]:
train_fin = pd.read_csv('train_fin.csv')
test_fin = pd.read_csv('test_fin.csv')
## target을 shift해서 현재의 feature로 다음 3시간을 예측할 수 있도록 설정
## 위에 순서까지 진행을 했으면 파일이 보존되어 있으므로 여기부터 시작

In [15]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
import xgboost as xgb

import math

In [16]:
'''
test_true : 실제 계측 값(list)
prediction_value : 모델이 예측한 값(list)
'''
## WRMSE 정의

def WRMSE(test_data, prediction_data):
    
    ## weight 계산
    unique_list = list(np.unique(test_data))
    weight_list = list()
    for count_number in unique_list:
        weight_list.append(test_data.count(count_number))
    
    ## weighted RMSE 계산
    wrmse_list = []
    temp_summation = 0
    for element in enumerate(prediction_data):
        temp_summation += (1/weight_list[unique_list.index(test_data[element[0]])])*pow(test_data[element[0]] - element[1], 2)
    wrmse_result = math.sqrt(temp_summation)
    
    return wrmse_result

In [17]:
index_list = train_fin.columns.tolist()
index_list.pop(index_list.index('target'))
feature_label = index_list
del index_list
target_label = ['target']

feature_data = train_fin[feature_label].values
target_data = train_fin[target_label].values
tscv = TimeSeriesSplit(max_train_size=None, n_splits=12)
print(tscv)

## Time Series Split 설정

TimeSeriesSplit(max_train_size=None, n_splits=12)


In [18]:
for train_index, test_index in tscv.split(feature_data):
    X_train, X_test = feature_data[train_index], feature_data[test_index]
    y_train, y_test = target_data[train_index], target_data[test_index]
    
    rfr = RandomForestRegressor()
    rfr.fit(X_train, y_train)
    y_preds = rfr.predict(X_test)

    WRMSE_true = list(map(lambda x : x[0], list(y_test)))
    y_preds = list(np.around(np.array(list(y_preds)),0))
    y_preds = list(map(lambda x : 0 if x <= 0 else x, y_preds))
    WRMSE_preds = list(map(lambda x : 9 if x >= 9 else x, y_preds))
    print(WRMSE(WRMSE_true, WRMSE_preds))

5.168692749442476
4.745529275684246
3.68442006245481
4.500364482634532
4.362461868887428
4.5704211761938645
3.9342959690375565
2.813375257550505
5.027560599003812
3.29784048247418
4.217434005883557
3.2497520550633245


In [19]:
for train_index, test_index in tscv.split(feature_data):
    X_train, X_test = feature_data[train_index], feature_data[test_index]
    y_train, y_test = target_data[train_index], target_data[test_index]
    
    model = xgb.XGBRegressor(objective ='reg:squarederror')
    model.fit(X_train, y_train)
    y_preds = model.predict(X_test)
    
    WRMSE_true = list(map(lambda x : x[0], list(y_test)))
    y_preds = list(np.around(np.array(list(y_preds)),0))
    y_preds = list(map(lambda x : 0 if x <= 0 else x, y_preds))
    WRMSE_preds = list(map(lambda x : 9 if x >= 9 else x, y_preds))
    print(WRMSE(WRMSE_true, WRMSE_preds))


5.192075176348891
4.239603519010367
3.4944893636335967
4.30994003120945
4.004120416580994
4.170375802476961
4.214024299132223
2.8549078905759866
4.8187138339488405
3.537550336390857
4.314361079543012
3.4209401558502868


In [20]:
model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [21]:
for train_index, test_index in tscv.split(feature_data):
    X_train, X_test = feature_data[train_index], feature_data[test_index]
    y_train, y_test = target_data[train_index], target_data[test_index]
    
    model = xgb.XGBRegressor(objective ='reg:squarederror', booster='gbtree', learning_rate=0.2, min_child_weight=1.5)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_test)
    
    WRMSE_true = list(map(lambda x : x[0], list(y_test)))
    y_preds = list(np.around(np.array(list(y_preds)),0))
    y_preds = list(map(lambda x : 0 if x <= 0 else x, y_preds))
    WRMSE_preds = list(map(lambda x : 9 if x >= 9 else x, y_preds))
    print(WRMSE(WRMSE_true, WRMSE_preds))


4.9779131153449505
4.3363675019371435
3.50841950144898
4.32609750940869
3.8291777945802434
3.9955059545544063
3.931601252510613
2.7698374576161293
4.886394023979587
3.4914335591048204
4.15470467303347
3.1864587514127667


In [22]:
for train_index, test_index in tscv.split(feature_data):
    X_train, X_test = feature_data[train_index], feature_data[test_index]
    y_train, y_test = target_data[train_index], target_data[test_index]
    
    model = xgb.XGBRegressor(objective ='reg:squarederror', booster='gbtree', learning_rate=0.15, min_child_weight=0.5,
                            max_depth=7)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_test)
    
    WRMSE_true = list(map(lambda x : x[0], list(y_test)))
    y_preds = list(np.around(np.array(list(y_preds)),0))
    y_preds = list(map(lambda x : 0 if x <= 0 else x, y_preds))
    WRMSE_preds = list(map(lambda x : 9 if x >= 9 else x, y_preds))
    print(WRMSE(WRMSE_true, WRMSE_preds))


5.612793047775386
5.240436826058216
3.5734414344850562
5.166631745367273
4.417699022668659
4.457813302807302
3.9565111646756983
2.73732797242002
4.812720078411192
3.6508338420720516
3.7530757802515504
3.0917429645752765


In [23]:
index_list = train_fin.columns.tolist()
index_list.pop(index_list.index('target'))
feature_label = index_list
del index_list
target_label = ['target']

feature_data = train_fin[feature_label].values
target_data = train_fin[target_label].values
tscv = TimeSeriesSplit(max_train_size=None, n_splits=12)
print(tscv)

## Time Series Split 설정

TimeSeriesSplit(max_train_size=None, n_splits=12)


In [24]:
for train_index, test_index in tscv.split(feature_data):
    X_train, X_test = feature_data[train_index], feature_data[test_index]
    y_train, y_test = target_data[train_index], target_data[test_index]
    
    model = xgb.XGBRegressor(objective ='reg:squarederror', booster='gbtree', learning_rate=0.2, min_child_weight=1.5)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_test)
    
    WRMSE_true = list(map(lambda x : x[0], list(y_test)))
    y_preds = list(np.around(np.array(list(y_preds)),0))
    y_preds = list(map(lambda x : 0 if x <= 0 else x, y_preds))
    WRMSE_preds = list(map(lambda x : 9 if x >= 9 else x, y_preds))
    print(WRMSE(WRMSE_true, WRMSE_preds))


4.9779131153449505
4.3363675019371435
3.50841950144898
4.32609750940869
3.8291777945802434
3.9955059545544063
3.931601252510613
2.7698374576161293
4.886394023979587
3.4914335591048204
4.15470467303347
3.1864587514127667


In [25]:
for train_index, test_index in tscv.split(feature_data):
    X_train, X_test = feature_data[train_index], feature_data[test_index]
    y_train, y_test = target_data[train_index], target_data[test_index]
    
    model = xgb.XGBRegressor(objective ='reg:squarederror', booster='gbtree', learning_rate=0.15, min_child_weight=0.5,
                            max_depth=7)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_test)
    
    WRMSE_true = list(map(lambda x : x[0], list(y_test)))
    y_preds = list(np.around(np.array(list(y_preds)),0))
    y_preds = list(map(lambda x : 0 if x <= 0 else x, y_preds))
    WRMSE_preds = list(map(lambda x : 9 if x >= 9 else x, y_preds))
    print(WRMSE(WRMSE_true, WRMSE_preds))


5.612793047775386
5.240436826058216
3.5734414344850562
5.166631745367273
4.417699022668659
4.457813302807302
3.9565111646756983
2.73732797242002
4.812720078411192
3.6508338420720516
3.7530757802515504
3.0917429645752765


> # 3. 결과 제출

In [26]:
test_fin.head()

,year,month,day,hour,mean_Np,mean_Tp,mean_Vp,mean_B_gsm_x,mean_B_gsm_y,mean_B_gsm_z,...,std_B_gsm_y,std_B_gsm_z,std_Bt,var_Np,var_Tp,var_Vp,var_B_gsm_x,var_B_gsm_y,var_B_gsm_z,var_Bt
0,2013,1,1,0,2.195024,22761.074691,353.657878,3.089253,-1.209734,-0.107792,...,0.364397,0.409743,0.175855,0.035094,1.333233e+07,4.868653,0.041541,0.132785,0.167890,0.030925
1,2013,1,1,3,2.556466,25284.098765,346.161446,3.257282,-1.025819,0.643181,...,0.217096,0.143583,0.051232,0.031385,8.598477e+06,6.748046,0.007467,0.047131,0.020616,0.002625
2,2013,1,1,6,2.429241,23017.538889,342.789659,3.080376,-1.340154,0.425454,...,0.276740,0.482064,0.085693,0.008152,1.083478e+07,1.592721,0.027214,0.076585,0.232386,0.007343
3,2013,1,1,9,2.101499,26302.041821,338.663912,3.096789,-1.192257,0.335477,...,0.219488,0.328673,0.086919,0.017724,8.301107e+06,5.343749,0.012425,0.048175,0.108026,0.007555
4,2013,1,1,12,2.124907,37699.452581,327.827076,2.594449,-1.279867,0.991676,...,0.283880,0.271309,0.096867,0.003107,2.946165e+07,5.671399,0.143226,0.080588,0.073609,0.009383


In [27]:
final_testset = test_fin[feature_label].values

In [28]:
final_pred = model.predict(final_testset)

In [29]:
final_pred = list(map(lambda x : x, list(final_pred)))
final_pred = list(np.around(np.array(list(final_pred)),0))
final_pred = list(map(lambda x : 0 if x <= 0 else x, final_pred))
final_pred = list(map(lambda x : 9 if x >= 9 else x, final_pred))

## 설정한 결과로 변환 완료

In [30]:
final_pred_list = list()
for i in range(int(len(final_pred)/8)):
    final_pred_list.append(final_pred[8*i:8*(i+1)])

In [31]:
tmp_result = pd.read_excel("Final_prediction.xlsx")

In [32]:
tmp_result.columns[1:]

Index(['kp_0h', 'kp_3h', 'kp_6h', 'kp_9h', 'kp_12h', 'kp_15h', 'kp_18h',
       'kp_21h'],
      dtype='object')

In [33]:
final_sheet = pd.DataFrame(columns=tmp_result.columns)
final_sheet['DOY'] = tmp_result['DOY']
final_sheet[tmp_result.columns[1:]] = pd.DataFrame(final_pred_list,columns=tmp_result.columns[1:]).astype(int)

> # 4. 결과 파일 생성

In [34]:
final_sheet.to_csv('HW_Final_prediction.csv', index=False)